<a href="https://colab.research.google.com/github/pasindu-pr/qdrant-q-and-a-on-pdf/blob/main/LLM%20Sherpa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install openai qdrant-client llmsherpa python-dotenv langchain langchain-community langchain-openai langchainhub openai tiktoken azure-ai-documentintelligence azure-identity azure-search-documents==11.4.0 pypdfium2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.9/165.9 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.1/166.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.8/283.8 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.4/193.4 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [3]:
from langchain import hub
from langchain_openai import AzureChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores.azuresearch import AzureSearch

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

from google.colab import userdata

os.environ["AZURE_OPENAI_ENDPOINT"] = userdata.get("AZURE_OPENAI_ENDPOINT")
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get("AZURE_OPENAI_API_KEY")
doc_intelligence_endpoint = userdata.get("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
doc_intelligence_key = userdata.get("AZURE_DOCUMENT_INTELLIGENCE_KEY")

In [5]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams
from qdrant_client.http import models
import openai
import os
from openai import AzureOpenAI

from google.colab import userdata


openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxx"

client = AzureOpenAI(
  api_key = userdata.get("AZURE_OPENAI_API_KEY"),
  api_version = "2024-02-01",
  azure_endpoint = userdata.get("AZURE_OPENAI_ENDPOINT")
)


In [ ]:
qdrant_client = QdrantClient(
    host=userdata.get('QDRANT_HOST'),
    api_key=userdata.get('QDRANT_KEY'),
)

In [ ]:
qdrant_client.recreate_collection(
    collection_name="mycollection",
    vectors_config=models.VectorParams(size=3072, distance=models.Distance.COSINE),
)
print("Create collection reponse:", qdrant_client)

Create collection reponse: <qdrant_client.qdrant_client.QdrantClient object at 0x7ce01b383220>


In [ ]:
collection_info = qdrant_client.get_collection(collection_name="mycollection")

print("Collection info:", collection_info)

Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=0 indexed_vectors_count=0 points_count=0 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=3072, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None) payload_schema={}


In [7]:
from llmsherpa.readers import LayoutPDFReader

llmsherpa_api_url = "https://readers.llmsherpa.com/api/document/developer/parseDocument?renderFormat=all"
pdf_url = "/content/InjuredWorkerGuidebookCalifornia.pdf" # also allowed is a file path e.g. /home/downloads/xyz.pdf
pdf_reader = LayoutPDFReader(llmsherpa_api_url)
doc = pdf_reader.read_pdf(pdf_url)

In [12]:
from qdrant_client.http.models import PointStruct

points=[]
chunks = []
i = 1
for chunk in doc.chunks():
    chunks.append(chunk.to_context_text())
    # i += 1
    # response = client.embeddings.create(
    #     input=chunk.to_context_text(),
    #     model="embedding-model"
    # )
    # embeddings = response.data[0].embedding

    # points.append(PointStruct(id=i, vector=embeddings, payload={"text": chunk.to_context_text()}))



In [ ]:
# Qdrant Indexing

operation_info = qdrant_client.upsert(
    collection_name="mycollection",
    points=points
)

print("Operation info:", operation_info)

In [13]:
chunks

['A Guidebook for Injured Workers > Sixth Edition April 2016\nThis guidebook was designed and produced by the Office of the Director, Department of Industrial Relations (DIR), in consultation with the Division of Workers’ Compensation (DWC), State of California.\nIt is based on the third edition of this guidebook, prepared in 2006 by the Institute for Research on Labor and Employment (IRLE) and the Labor Occupational Health Program, University of California, Berkeley, for the Commission on Health and Safety and Workers’ Compensation, DIR.',
 'A Guidebook for Injured Workers > Sixth Edition April 2016\nSixth revised edition, copyright © 2016 California Division of Workers’ Compensation.\nAll or portions of this guidebook may be reproduced without permission for educational purposes.\nPlease credit DIR and DWC.',
 'A Guidebook for Injured Workers > Sixth Edition April 2016\nThe guidebook can be viewed and downloaded at www.dir.ca.gov, www.dwc.ca.gov, and www.dir.ca.gov/chswc/.\nMany publ

In [20]:
# Azure Search Indexing

model = userdata.get("EMBEDDING_MODEL")
endpoint = userdata.get("AZURE_SEARCH_ENDPOINT")
adminkey = userdata.get("AZURE_SEARCH_ADMIN_KEY")

embeddings = AzureOpenAIEmbeddings(model=model,deployment="embedding-model",
                                   azure_endpoint=userdata.get("AZURE_OPENAI_ENDPOINT"),
                                  chunk_size=1)

vector_store_address: str = endpoint
vector_store_password: str = adminkey

index_name: str = "test"

vector_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

res = vector_store.add_texts(chunks)
print(res)

['...']


### Get Chunks

In [23]:

# Retrieve relevant chunks based on the question

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3})

retrieved_docs = retriever.get_relevant_documents(
    "Examples of Temporary Total Disability Benefits"
)

print(retrieved_docs)

print(userdata.get("CHAT_MODEL"))

chat_model = userdata.get("CHAT_MODEL")

# Use a prompt for RAG that is checked into the LangChain prompt hub (https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=989ad331-949f-4bac-9694-660074a208a7)
prompt = hub.pull("rlm/rag-prompt")
llm = AzureChatOpenAI(
    openai_api_version="2023-05-15",
    azure_deployment=chat_model,
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

[Document(page_content='Chapter 5. Temporary Disability Benefits > Notices About TD Payments > Temporary Total Disability Payments—Some Examples\n | Baker & Janitor (1 person with 2 jobs) | 2015 | $620 per week (baker) $280 per week (janitor) $900 per week total | $600 per week (if the injury was caused by the higher paying job) | This is two-thirds of the wages from both jobs combined\n | --- | --- | --- | --- | ---\n | Truck Driver | 2016 | $1,300 per week | $866.67 per week | This is two-thirds of the wages earned before injury\n | Accountant | 2016 | $1,800 per week | $1,128.43 per week | This is the maximum for workers injured in 2016\n'), Document(page_content='A Guidebook for Injured Workers > Workers’ Compensation Benefits—Examples > Temporary Total Disability Benefits\n | DATE OF INJURY | MINIMUM PAYMENTS | MAXIMUM PAYMENTS\n | --- | --- | ---\n | 2010 | $148.00 per week | $986.69 per week\n | 2011 | $148.00 per week | $986.69 per week\n | 2012 | $151.57 per week | $1,010.50 p

In [24]:
rag_chain.invoke("Examples of Temporary Total Disability Benefits")

'Temporary total disability benefits are payments given to injured workers who are unable to work during their recovery. Examples of TTD payments include $900 per week for a person with two jobs, $1,300 per week for a truck driver, and $1,800 per week for an accountant. The amount of TTD payments varies depending on the date of injury, with a minimum of $148 per week in 2010 and a maximum of $1,128.43 per week in 2016.'

## Qdrant Indeixng Start Point

In [ ]:
operation_info = qdrant_client.upsert(
    collection_name="mycollection",
    points=points
)

print("Operation info:", operation_info)

In [ ]:
def create_answer_with_context(query):
    response = client.embeddings.create(
        input="What is the GROSS WAGES BEFORE INJURY for different occupations?",
        model="embedding-model"
    )
    embeddings = response.data[0].embedding

    search_result = qdrant_client.search(
        collection_name="mycollection",
        query_vector=embeddings,
        limit=5
    )

    prompt = "Context:\n"
    for result in search_result:
        prompt += result.payload['text'] + "\n---\n"
    prompt += "Question:" + query + "\n---\n" + "Answer:"

    print("----PROMPT START----")
    print(":", prompt)
    print("----PROMPT END----")

    completion = client.chat.completions.create(
        model=userdata.get("CHAT_MODEL"),
        messages=[
            {"role": "user", "content": prompt}
        ]
        )

    return completion.choices[0].message.content



In [ ]:

input = "What is the GROSS WAGES BEFORE INJURY for different occupations"
answer = create_answer_with_context(input)
print(answer)
